<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# получение общего количества вакансий в таблице
query_3_1 = f'''
select
    count(id)
from vacancies
'''

In [5]:
df_3_1 = pd.read_sql_query(query_3_1, connection)

display(f'Количество вакансий: {df_3_1.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\1827617512.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_1 = pd.read_sql_query(query_3_1, connection)


'Количество вакансий: 49197.'

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# получение количества уникальных работодателей в таблице с помощью DISTINCT
query_3_2 = f'''
select
    count(distinct(id))
from employers
'''

In [7]:
df_3_2 = pd.read_sql_query(query_3_2, connection)

display(f'Количество работодателей: {df_3_2.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3893326725.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_2 = pd.read_sql_query(query_3_2, connection)


'Количество работодателей: 23501.'

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# получение количества уникальных регионов в таблице с помощью DISTINCT
query_3_3 = f'''
select
    count(distinct(id))
from areas
''' 

In [9]:
df_3_3 = pd.read_sql_query(query_3_3, connection)

display(f'Количество регионов: {df_3_3.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3147658958.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_3 = pd.read_sql_query(query_3_3, connection)


'Количество регионов: 1362.'

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# получение количества уникальных сфер деятельности с помощью DISTINCT
query_3_4 = f'''
select
    count(distinct(id))
from industries
'''

In [11]:
df_3_4 = pd.read_sql_query(query_3_4, connection)

display(f'Количество сфер деятельности: {df_3_4.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3529499414.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_4 = pd.read_sql_query(query_3_4, connection)


'Количество сфер деятельности: 294.'

***

После предварительного анализа данные можно сделать некоторые выводы:
* Количество вакансий составляет 49197, колчиество работодателей - 23501. В среднем каждый работодатель предлагает на рынке труда 2 вакансии;
* Охват вакансий достаточно обширный - 1362 региона и 294 сферы деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# запрос, позволяющий узнать количество вакансий в каждом регионе
# к таблице с регионами с помощью LEFT JOIN присоединим таблицу с вакансиями
# сгруппируем по регионам и посчитаем количества вакансий в каждом из них
query_4_1 = f'''
select
    a.name area,
    count(*) cnt
from areas a
    join vacancies v on v.area_id = a.id
group by a.id
order by 2 desc
'''

In [13]:
df_4_1 = pd.read_sql_query(query_4_1, connection)

display(df_4_1)

C:\Users\User\AppData\Local\Temp\ipykernel_15332\2469922489.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_1 = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# подсчет количества вакансий с указанием хотя бы одного поля с зарплатой
# фильтруем поля с помощью их проверки на содержание значения NULL, а также оператора OR 
query_4_2 = f'''
select
    count(*)
from vacancies
where salary_from is not Null
    or salary_to is not Null
'''

In [15]:
df_4_2 = pd.read_sql_query(query_4_2, connection)

display(f'Количество вакансий, в которых указано хотя бы одно поле с зарплатой, составляет {df_4_2.loc[0, "count"]}')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\4021686524.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_2 = pd.read_sql_query(query_4_2, connection)


'Количество вакансий, в которых указано хотя бы одно поле с зарплатой, составляет 24073'

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# запрос, выводящий зарплатную вилку (нижняя и верхняя границы зарплаты)
# значения получаем, прияменяя к колонкам salary_from и salary_to агрегатную функцию AVG (среднее значение)
query_4_3 = f'''
select
    round(avg(salary_from)) down,
    round(avg(salary_to)) up
from vacancies
'''

In [17]:
df_4_3 = pd.read_sql_query(query_4_3, connection)
down, up = df_4_3.loc[0]

display(f'Нижняя граница зарплатной вилки - {down} р., верхняя - {up} р.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3077443925.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_3 = pd.read_sql_query(query_4_3, connection)


'Нижняя граница зарплатной вилки - 71065.0 р., верхняя - 110537.0 р.'

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# получение сочетаний типа рабочего графика и типа трудоустройства
# группировка по типа рабочего графика и типу трудоустройства
# и дальнейший подсчет количества получившихся комбинаций
query_4_4 = f'''
select
    format('%s - %s', schedule, employment) "Schedule - Employment",
    count(*) cnt
from vacancies
group by schedule, employment
order by 2 desc
'''

In [19]:
df_4_4 = pd.read_sql_query(query_4_4, connection)
second_place = df_4_4.iloc[1, 0]

display(df_4_4)
display(f'На втором месте по популярности находится сочетание "{second_place}"')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\887409054.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_4 = pd.read_sql_query(query_4_4, connection)


,Schedule - Employment,cnt
0,Полный день - Полная занятость,35367
1,Удаленная работа - Полная занятость,7802
2,Гибкий график - Полная занятость,1593
3,Удаленная работа - Частичная занятость,1312
4,Сменный график - Полная занятость,940
5,Полный день - Стажировка,569
6,Вахтовый метод - Полная занятость,367
7,Полный день - Частичная занятость,347
8,Гибкий график - Частичная занятость,312
9,Полный день - Проектная работа,141


'На втором месте по популярности находится сочетание "Удаленная работа - Полная занятость"'

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# получение количества вакансий для кажого значения в графе "Требуемый опыт работы"
# группировка значений по опыту работы, дальнейший подсчет количества для каждого значения
# и сортировка в порядке возрастания
query_4_5 = f'''
select
    experience,
    count(*)
from vacancies
group by experience
order by 2
'''

In [21]:
df_4_5 = pd.read_sql_query(query_4_5, connection)

display(df_4_5)

C:\Users\User\AppData\Local\Temp\ipykernel_15332\2719498419.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_5 = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Проведя анализ вакансий, можно сделать следующие выводы:
* Наибольшее количество вакансий представлено в Москве, Санкт-Петербурге, Минске, Новосибирске и Алматы. Все представленные города - миллионники, поэтому такое количество вакансий не удивительно;
* В среднем ожидаемая заработная плата находится интервале от 71 тыс. руб до 110 тыс.руб.;
* Наибольшее количество вакансий представлено для удаленного типа рабочего графика на полную занятость. Возможно, это связано с последствиями пандемии короновирусной инфекции, когда многие работодатели переводили своих сотрудников на удаленную работу на полный рабочий день. Из позитивных сторон такого перехода можно выделить сокращение расходов на аренду помещений, офисов, электроэнергию и т.п., так как основной штат работает из дома; сокращение не только расходов сотрудников на дорогу до работы, но и исключение временных затрат на это. Можно предположить, что для работодателей такой вариант оказался на руку, отсюда и популярность такого сочетания.
* Наибольшее количество вакансий представлено для соискателей с опытом работы от 1 до 3 лет. В такую категорию попадает в основном молодежь, которая наиболее активная, смелая, амбициозная, креативная и идет в ногу со временем. Возможно, работодатели придерживаются плана "выловить" перспективных начинающих специалистов и раскрыть их потенциал, что в дальнейшем принесет свои плоды.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# подсчет количества вакансий, которое предлагает каждый работодатель
# к таблице с работодателями с помощью LEFT JOIN присоединяем таблицу с вакансиями
# группируем полученную таблицу по работодателям и подсчитываем количество вакансий для каждого
# сортируем в порядке убывания и составляем ТОП-5 с помощью LIMIT
query_5_1 = f'''
select
    e.name employer,
    count(*) cnt
from employers e
    join vacancies v on v.employer_id = e.id
group by e.id
order by 2 desc
limit 5
'''

In [23]:
df_5_1 = pd.read_sql_query(query_5_1, connection)

display(df_5_1)
display(f'На первом месте по кол-ву вакансий находится {df_5_1.loc[0, "employer"]} с {df_5_1.loc[0, "cnt"]} вакансиями(ей), на пятом - {df_5_1.loc[4, "employer"]} с {df_5_1.loc[4, "cnt"]} вакансями(ей).')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\681184114.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_1 = pd.read_sql_query(query_5_1, connection)


,employer,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


'На первом месте по кол-ву вакансий находится Яндекс с 1933 вакансиями(ей), на пятом - Газпром нефть с 331 вакансями(ей).'

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# подсчет количества работодателй и вакансий в каждом регионе
# к таблице с регионами с помощью LEFT JOIN присоединяем таблицы с работодателями и вакансиями
# оставляем только те строки, в которых нет вакансий - ячейка id вакансии содержит значение NULL
# группируем по регионам, подсчитываем количество работодателей в каждом из них и выводим в порядке убывания количества работодателей
query_5_2 = f'''
select
    a.name region,
    count(distinct e.id) employers
from areas a
    left join employers e on a.id = e.area
    left join vacancies v on v.area_id = a.id
where v.id is null
group by a.id
order by 2 desc
'''

In [25]:
df_5_2 = pd.read_sql_query(query_5_2, connection)

display(f'Среди регионов, в которых нет вакансий, первое место по количеству работодателей занимает регион {df_5_2.loc[0, "region"]} - {df_5_2.loc[0, "employers"]} работодателя(ей).')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\760490785.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_2 = pd.read_sql_query(query_5_2, connection)


'Среди регионов, в которых нет вакансий, первое место по количеству работодателей занимает регион Россия - 410 работодателя(ей).'

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# подсчет количества регионов для каждого работодателя, в которых он публикует свои вакансии
# к таблице с работодателями с помощью LEFT JOIN присоединяем таблицу с вакансиями
# группируем полученную таблицу по работодателям и подсчитываем количество уникальных регионов, в которых работодатель опубликовал вакансии
# сортируем по количеству вакансий по убыванию и выводим 
query_5_3 = f'''
select
    e.name employer,
    count(distinct(v.area_id)) cnt_areas
from employers e
    left join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
'''

In [27]:
df_5_3 = pd.read_sql_query(query_5_3, connection)

display(df_5_3)
display(f'{df_5_3.loc[0, "employer"]} публикует свои вакансии в наибольшем количестве регионов - {df_5_3.loc[0, "cnt_areas"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\206146905.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_3 = pd.read_sql_query(query_5_3, connection)


,employer,cnt_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


'Яндекс публикует свои вакансии в наибольшем количестве регионов - 181.'

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# подсчет количества работодателей, у которых не указана сфера деятельности
# к таблице с работодателями с помощью LEFT JOIN присоединяем таблицу, которая связывает id работодателей и id сферы деятельности
# фильтруем таблицу, оставляя лишь те строки, в которых указан id сферы деятельности (ячечка сферы деятельности не равна NULL)
# подсчитываем уникальное колчиество оставшихся работодателей
query_5_4 = f'''
select
    count(distinct(e.id)) cnt_employers
from employers e
    left join employers_industries ei on e.id = ei.employer_id
where ei.industry_id is null
'''

In [29]:
df_5_4 = pd.read_sql_query(query_5_4, connection)

display(f'Количество работодателей, у которых не указана сфера деятельности: {df_5_4.loc[0, "cnt_employers"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\2349454416.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_4 = pd.read_sql_query(query_5_4, connection)


'Количество работодателей, у которых не указана сфера деятельности: 8419.'

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# выводим название компании, которая находится на 3 месте в алфавитном списке компаний, у которых указано 4 сферы деятельности
# к таблице с работодателями с помощью LEFT JOIN присоединяем таблицу, которая связываем id работодателей и id сферы деятельности
# группируем полученную таблицу по работодателям и с помощью DISTINCT подсчитываем количество уникальных сфер деятельности для каждого работодателя
# с помощью HAVING фильтруем, осталяя тех работодателей, у которых уникальное количество сфер деятельности равно 4
# сортируем работодателей по алфавиту и с помощью OFFSET и LIMIT выводит третью компанию по списку
query_5_5 = f'''
select
    e.name employer,
    count(distinct(ei.industry_id)) cnt
from employers e
    left join employers_industries ei on e.id = ei.employer_id
group by e.id
having count(distinct(ei.industry_id)) = 4
order by 1
offset 2 limit 1
'''

In [31]:
df_5_5 = pd.read_sql_query(query_5_5, connection)

display(f'Компания, находящаяся на третьем месте в алфавитном списке компаний, у которых указано 4 сферы деятельности, это {df_5_5.loc[0, "employer"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3756331230.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_5 = pd.read_sql_query(query_5_5, connection)


'Компания, находящаяся на третьем месте в алфавитном списке компаний, у которых указано 4 сферы деятельности, это 2ГИС.'

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# подсчет количества работодателей, занимающихся разработкой программного обеспечения
# к таблице с работодателями с помощью LEFT JOIN присоединяем таблицу, которая связываем id работодателей и id сферы деятельности,
# а также таблицу, в которой указаны id и название сферы деятельности
# фильтруем таблицу, оставляя лишь те строки, в которых название сферы деятельности относится к разработке программного обеспечения
# с помощью DISTINCT подсчитываем количество уникальных работодателей 
query_5_6 = f'''
select
    count(distinct(e.id))
from employers e
    left join employers_industries ei on e.id = ei.employer_id
    left join industries i on ei.industry_id = i.id
where i.name = 'Разработка программного обеспечения'
'''

In [33]:
df_5_6 = pd.read_sql_query(query_5_6, connection)

display(f'"Разработка программного обеспечения" как сфера деятельности указана у {df_5_6.loc[0, "count"]} работодателя(ей).')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\2006296335.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_6 = pd.read_sql_query(query_5_6, connection)


'"Разработка программного обеспечения" как сфера деятельности указана у 3553 работодателя(ей).'

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# получаем список городов-миллионников с сайта Википедия с помощью requests и pd.read_html
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров'
cities = tuple(pd.read_html(requests.get(url).text)[1]['Город'])

In [35]:
# список городов-миллионников, в которых для каждого представлено количество вакансии компании Яндекс, а также суммарное количество вакансий в этих городах
# к таблице с регионами с помощью LEFT JOIN присоединим таблицу с вакансиями, а также таблицу с работодателями
# отфильтруем получившуюся таблицу, оставив лишь те строки, где работодатель - Яндекс, и город, в котором размещена вакансия, находится в кортеже с городами-миллионниками
# сгруппируем таблицу по городам и подсчитаем количество вакансий для каждого города
# далее с помощью UNION к полученной таблице добавляем строку, в которой указано суммарное количество вакансий компании Яндекс в городах-миллионниках
# сортируем полученную таблицу по возрастанию количества вакансий в городах и выводим
query_5_7 = f'''
select
    a.name city,
    count(v.id) cnt
from areas a
    left join vacancies v on a.id = v.area_id
    left join employers e on v.employer_id = e.id
where a.name in {cities} and e.name = 'Яндекс'
group by a.name

union

select
    'total',
    sum(x.cnt)
from
(
select
    a.name city,
    count(v.id) cnt
from areas a
    left join vacancies v on a.id = v.area_id
    left join employers e on v.employer_id = e.id
where a.name in {cities} and e.name = 'Яндекс'
group by a.name
) as x
order by 2
'''

In [36]:
df_5_7 = pd.read_sql_query(query_5_7, connection)
display(df_5_7)

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3666685428.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_7 = pd.read_sql_query(query_5_7, connection)


,city,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

По результатам анализа работодателей можно сделать следующие выводы:
* У компании "Яндекс" больше всего размещенных вакансий - 1933. "Яндекс" крупная компания, занимающая приличную часть рынка в IT сфере, сфере доставки и маркетплейса, каршеринга, такси, спутниковых карт (Яндекс.Еда, Яндекс.GO, Яндекс.Маркет и т.д.). К тому же у компании самый большой охват по регионам - вакансии компании "Яндекс" размещены в 181 регионе. Причем четверть вакансий представлена в городах-миллиониках (485 из 1933);
* Наибольшее количество вакансий размещено в России;
* Cфера деятельности не указана у 35% работодателей (8419 из 23501);
* 15% работодателей занимаются разработкой программного обеспечения (3553 из 23501).

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# подсчет вакансий, имеющих отношение к данным
# фильтруем таблицу с вакансиями, оставляем только те строки, которые удовлетворяют условию, и подсчитываем общее количество вакансий
query_6_1 = f'''
select
    count(*) count
from vacancies
where lower(name) like '%data%' or lower(name) like '%данн%'
'''

In [38]:
df_6_1 = pd.read_sql_query(query_6_1, connection)
display(f'Количество вакансий, имеющих отношения к данным, равно {df_6_1.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\760168937.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_1 = pd.read_sql_query(query_6_1, connection)


'Количество вакансий, имеющих отношения к данным, равно 1771.'

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# подсчет ваканский для начинающего дата-саентиста
# фильтруем таблицу с вакансиями, оставляем лишь те строки, которые удовлетворяют условию, и подсчитваем количество вакансий
query_6_2 = f'''
select
    count(*) count
from vacancies
where (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    and (lower(name) like '%junior%'
    or experience = 'Нет опыта'
    or employment = 'Стажировка')
'''

In [40]:
df_6_2 = pd.read_sql_query(query_6_2, connection)
display(f'Количество вакансий, подходящих для начинающего дата-саентиста, равно {df_6_2.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\733136966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_2 = pd.read_sql_query(query_6_2, connection)


'Количество вакансий, подходящих для начинающего дата-саентиста, равно 51.'

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# подсчет ваканский, где в качетсве ключевого навыка указан SQL или Postgres
# фильтруем таблицу с вакансиями, оставляем лишь те строки, которые удовлетворяют условию, и подсчитываем количество вакансий
query_6_3 = f'''
select
    count(*) count
from vacancies
where (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    and (lower(key_skills) like '%sql%' or lower(key_skills) like '%postgres%')
'''

In [42]:
df_6_3 = pd.read_sql_query(query_6_3, connection)
display(f'Количество вакансий для дата-саентиста, в которых в качестве ключевого навыка указан SQL или Postgres, составляет {df_6_3.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3760088221.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_3 = pd.read_sql_query(query_6_3, connection)


'Количество вакансий для дата-саентиста, в которых в качестве ключевого навыка указан SQL или Postgres, составляет 201.'

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# подсчет ваканский, где в качетсве ключевого навыка указан Python
# фильтруем таблицу с вакансиями, оставляем лишь те строки, которые удовлетворяют условию, и подсчитываем количество вакансий
query_6_4 = f'''
select
    count(*) count
from vacancies
where (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    and lower(key_skills) like '%python%'
'''

In [44]:
df_6_4 = pd.read_sql_query(query_6_4, connection)
display(f'Количество ваканский, в которых в качестве ключевого навыка указан Python, составляет {df_6_4.loc[0, "count"]}.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\1308678084.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_4 = pd.read_sql_query(query_6_4, connection)


'Количество ваканский, в которых в качестве ключевого навыка указан Python, составляет 351.'

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# подсчет среднего количества ключевых навыков в вакансиях для дата-саентиста
# фильтруем таблицу с вакансиями, оставляем лишь те вакансии, которые подходят для дата-саентистов
# количество ключевых навыков находим следующим образом: вычисляем длину строки в разделе "Ключевые навыки",
# затем измеряем длину строки, в которой символ табуляции заменен на пустой символ ""
# находим разницу длин строк до замены и после и прибавляем к разности 1
query_6_5 = f'''
select
    round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2) cnt
from vacancies
where (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
'''

In [46]:
df_6_5 = pd.read_sql_query(query_6_5, connection)
display(f'В среднем в вакансиях указывают {df_6_5.loc[0, "cnt"]} ключевых навыка.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\539117262.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_5 = pd.read_sql_query(query_6_5, connection)


'В среднем в вакансиях указывают 6.41 ключевых навыка.'

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# вычисление средней заработной платы для дата-саентиста для каждого типа требуемого опыта работы
# фильруем таблицу с вакансиями, оставляем лишь те строки, которые удовлетворяют условию, а также те, в которых указана хотя бы одна из зарплат
# группируем по опыту работы и с помощью функции COALESCE выводим значение средней заработной платы, если заполнены оба поля с зарплатой,
# или же значение той заработной платы, которая указана
query_6_6 = f'''
select
    experience,
    round(avg(coalesce((salary_from + salary_to) / 2, salary_from, salary_to))) salary
from vacancies
where (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    and (salary_from is not null or salary_to is not null)
group by experience
'''

In [48]:
df_6_6 = pd.read_sql_query(query_6_6, connection)
exp, salary = df_6_6[df_6_6['experience'] == 'От 3 до 6 лет'].iloc[0]

display(df_6_6)
display(f'Средняя зарплата для дата-саентиста с опытом работы {exp} составляет {salary} рублей.')

C:\Users\User\AppData\Local\Temp\ipykernel_15332\3592129786.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_6 = pd.read_sql_query(query_6_6, connection)


,experience,salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


'Средняя зарплата для дата-саентиста с опытом работы От 3 до 6 лет составляет 243115.0 рублей.'

***

Проведя предметный анализ, можно выделить следующее:
* Из 1771 вакансии, имеющей отношение к данным, лишь 51 вакансия подходит для начинающего дата-саентиста, что составляет около 3%. Похоже, что начинающему дата-саентисту будет сложно найти работу - в приоритете кандидаты с опытом работы;
* Из 1771 вакансии, имеющей отношение к данным, только в 351 вакансии указан Python как ключевой навык, что составляет около 20%. Знание же SQL или Postrges как ключевой навык указаны в 201 вакансии. Это может говорить о том, что для работы с данными есть множество других немаловажных инструментов и программ, что подтверждают следующие выводы - в среднем в вакансиях по работе с данными работодатели указывают около 6 ключевых навыков;
* Самая низкая средняя заработная плата среди ваканкий, имеющих отношение к данным, представлена для кандидатов без опыта работы - около 74 тыс.руб., самая высокая - для кандидатов с опытом работы от 3 до 6 лет - около 243 тыс.руб. Такие данные вполне ожидаемы.

# Общий вывод по проекту

Проведем дополнительные исследования: рассмотрим среднюю заработную плату, которую предлагают по каждой сфере деяьтельности, а также выведем 10 самых популярных сфер деятельности, которые встречаются в вакансиях, в процентном соотношении от их общего количества.

Расмотрим среднюю заработную плату по каждой сфере деятельности. В расчете будем использовать среднюю заработную плату. Метод подсчета аналогичен разделу 6.6: если заполнены оба поля с зарплатой, то считаем среднюю, если одно из двух полей, то зарплатой считаем его. Вакансии с двумя пустыми полями в расчет не берем.

In [49]:
# К таблице с вакансиями методом JOIN присоединяем таблицу с работодателями, таблицу, связывающую работодателя и сферу деятельности,
# а также таблицу с вариантами сфер деятельности.
# С помощью метода WHERE исключаем вакансии, в которых не заполнено ни одно поле с зарплатой.
# С помощью COALESCE считаем среднюю зарплату для каждой вакансии.
# С помощью GROUP BY группируем таблицу по сферам деятельности и находим среднюю зарплату для каждой сферы деятельности.
# Сортируем по убыванию средней зарплаты.

query_7_1 = f'''
select
    i.name industry,
    round(avg(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to))) mean_salary
from vacancies v
    join employers e on v.employer_id = e.id
    join employers_industries ei on e.id = ei.employer_id
    join industries i on ei.industry_id = i.id
where v.salary_from is not null
    or v.salary_to is not null
group by i.id
order by 2 desc
'''

In [50]:
df_7_1 = pd.read_sql_query(query_7_1, connection)

most_expensive_industry_name = df_7_1.iloc[0, 0]
most_expensive_industry_mean_salary = df_7_1.iloc[0, 1]

most_cheap_industry_name = df_7_1.iloc[-1, 0]
most_cheap_industry_mean_salary = df_7_1.iloc[-1, 1]

display(f'Самая высокая средняя заработная плата наблюдается в сфере "{most_expensive_industry_name}", которая составляет {most_expensive_industry_mean_salary} рублей')
display(f'Самая низкая средняя заработная плата наблюдается в сфере "{most_cheap_industry_name}", которая составляет {most_cheap_industry_mean_salary} рублей')
display('Посмотрим на все сферы деятельности:')
display(df_7_1)

C:\Users\User\AppData\Local\Temp\ipykernel_15332\2362288548.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_7_1 = pd.read_sql_query(query_7_1, connection)


'Самая высокая средняя заработная плата наблюдается в сфере "Агрохимия (продвижение, оптовая торговля)", которая составляет 381429.0 рублей'

'Самая низкая средняя заработная плата наблюдается в сфере "Детское питание (производство)", которая составляет 32868.0 рублей'

'Посмотрим на все сферы деятельности:'

,industry,mean_salary
0,"Агрохимия (продвижение, оптовая торговля)",381429.0
1,Агрохимия (производство),339875.0
2,Игорный бизнес,191905.0
3,Корма для животных (производство),168300.0
4,Зоомагазин,167613.0
...,...,...
286,Фото- и видеоуслуги,40961.0
287,Ритуальные услуги,37401.0
288,"Архив, библиотека, искусствоведение",36858.0
289,Переработка газа,34068.0


Теперь выделим 10 самых популярных сфер деятельности среди представленных вакансий.

In [51]:
# К таблице с вакансиями с помощью JOIN добавим таблицу с работодателями, таблицу, связывающую работодателя и сферу деятельности,
# а также таблицу с вариантами сфер деятельности.
# Оставим столбец с названием сфер деятельности. 

query_7_2 = f'''
select
    i.name industry
from vacancies v
    join employers e on v.employer_id = e.id
    join employers_industries ei on e.id = ei.employer_id
    join industries i on ei.industry_id = i.id
'''

In [52]:
df_7_2 = pd.read_sql_query(query_7_2, connection)

# Найдем уникальное количество сфер деятельности, указанных в представленных вакансиях
industries_count = len(df_7_2['industry'].value_counts())
display(f'Количество уникальных сфер деятельности, указанных в представленных вакансиях, равно {industries_count}')

# С помощью метода "value_counts()" найдем сколько раз встречается каждая сфера деятельности
# и с помощью метода "nlargest()" выделим 10 самых популярных в отдельную серию
popular = df_7_2['industry'].value_counts().nlargest(10)

# Чтобы на графике не выводить все сферы деятельности, а сосредоточиться на тех, которые нас интересуют - на популярных,
# заменим названия сфер деятельности, не входящих в список популярных, на "other"
df_7_2['industry'] = df_7_2['industry'].apply(lambda x: x if x in popular else 'other')

# Создадим серию, в которой посчитаем количество популярных сфер деятельности и тех, которые в это список не попали
df = round(df_7_2['industry'].value_counts(normalize=True).sort_values(ascending=False) * 100, 1)

C:\Users\User\AppData\Local\Temp\ipykernel_15332\2053629280.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_7_2 = pd.read_sql_query(query_7_2, connection)


'Количество уникальных сфер деятельности, указанных в представленных вакансиях, равно 293'

In [53]:
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [54]:
# С помощью Plotly выведем круговую диаграмму, в которой выразим долю популярных сфер деятельности от числа всех вакансий
industries_pie = go.Figure(data=[go.Pie(labels=list(df.index), values=list(df.values))])

industries_pie.update_layout(
    height=600,
    width=1500,
    title={
        'text': 'ТОП-10 сфер деятельности в процентном соотношении от общего количества вакансий',
        'x': 0.5,
        'xanchor': 'center'
    },
    title_font={
        'size': 30
    },
    annotations=[dict(
        text='Industries',
        x=0.5,
        y=0.5,
        font_size=15,
        showarrow=False
    )]
)

industries_pie.update_traces(hole=0.4, hoverinfo='label')

industries_pie.show()
industries_pie.write_html('images/industries.html')

Самой популярной сферой деятельности является "Разработка программного обеспечения", доля которой составляет 15.6% от общего количества вакансий. К тому же, ТОП-5 сфер деятельности относятся к информационным технологиям, банкам и мобильной связи, суммарно они составляют 42.8% от общего количества вакансий. Также можно заметить, что ТОП-10 сфер деятельности занимает 49.6%, половину от общего количества вакансий. Остальные 50.4% вакансий приходятся на 283 сферы деятельности. Возможно компании стараются активно внедрять в бизнес-процессы информационные технологии: разработка собственных приложений, сайтов, продвижение в социальных сетях, реклама продукции и т.д. (популярность среди вакансий таких сфер деятельности как "Разработки программного обеспечения", "Системной интеграции, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг" и "Интернет-компании").

В целом по проекту можно сделать следующие выводы:
* В среднем работодатель предлагает 2 вакансии;
* Наибольшее количество вакансий представлено России; в распределении по регионам - в столицах и городах-миллионниках;
* Больше всего вакансий для кандидатов с опытом работы от 1 до 3 лет;
* Самая популяная пара "Тип рабочего графика - тип трудоустройства" это "Удаленная работа - полная занятость";
* Больше всего вакансий размещает компания "Яндекс";
* 35% работодателей по какой-то причине не указали сферу деятельности;
* Анализ вакансий, имеющих отношение к данным, показал, что для начинающих дата-саентистов подходит 2% из этих вакансий; в среднем указывают около 6 ключевых навыков;
* Наибольшая зарплата в сфере "Агрохимия (продвижение, оптовая торговля)" - 381429 рублей, наименьшая в сфере "Детское питание (производство)" - 32868 рублей.
* Самой популярной сферой деятельности является "Разработка программного обеспечения" - 15.6% вакансий;
* ТОП-10 популярных сфер деятельности занимают половину от всех вакансий.

In [55]:
connection.close()